In [48]:
def frac(x):
    return(x-ZZ(floor(x)))

Given $\omega\in F$ a real quadratic number, $c\geq0,d\in\mathbb{Z}^2$, let $\beta=c\omega+d$, we want to compute
\begin{equation}
\begin{split}
h_\omega(t,c,d,v)&=\sum_{j=0}^{c-1}f_t\biggl(\biggl\{v_1-\frac{d}{c}(j+v_2)\biggl\}\biggl)f_{\beta t}\biggl(\frac{j+\{v_2\}}{c}\biggl)
\\
=&\sum_{j=0}^{c-1}\frac{e^{t\{v_1-\frac{d}{c}(j+v_2)\}}}{(e^t-1)}\frac{e^{\beta t\bigl(\frac{j+\{v_2\}}{c}\bigl)}}{(e^{\beta t}-1)},
\end{split}
\end{equation}
We first compute independently 

$$tf_t\biggl(\biggl\{v_1-\frac{d}{c}(j+v_2)\biggl\}\biggl)= \frac{te^{t\{v_1-\frac{d}{c}(j+v_2)\}}}{e^t-1}
\,\,\,\textup{and }\,\,\,
tf_{ t}\biggl(\frac{j+\{v_2\}}{c}\biggl)=\frac{te^{ t\frac{j+\{v_2\}}{c}}}{e^{ t}-1}.$$

As polynomials in $\mathbb{Q}$, and then we remark that 

$$
    f_t\biggl(\biggl\{v_1-\frac{d}{c}(j+v_2)\biggl\}\biggl) f_{\beta t}\biggl(\frac{j+\{v_2\}}{c}\biggl)[t^k]=\sum_{j=-1}^{k+1}\beta^jf_t\biggl(\frac{j+\{v_2\}}{c}\biggl)[t^j]f_t\biggl(\biggl\{v_1-\frac{d}{c}(j+v_2)\biggl\}\biggl) [t^{k-j}].
$$

To stay in $F[[t]]$ before the smoothing, we actually compute $t^2h_\omega(t,c,d,v)$, this explain that you will see an offset of 2 when we take the coeffcients of $h_\omega(t,c,d,v)$ in the code below.

In [50]:
def power_series_values_for_sum(c,d,order,v1,v2):##we dont put beta to optimize the computation
    Pol.<u> = QQ[]   
    L1=[]
    L2=[]
    den = u._exp_series(order) >> 1 # shift (P-P(0))/u
    invden = den.inverse_series_trunc(order) ## use the flint library
    for j in range(0,c):  
        x1 = Pol.base_ring()((j+frac(v2))/c) 
        num1 = (x1*u)._exp_series(order) #_exp_series result as a polynomial
        res1 = num1.multiplication_trunc(invden, order)
        if frac(((j+frac(v2))/c))==0:## we change the value of b_1(0), this doesnt change antyhing for the computation of log_p(u_l(omega)), but is needed for ord_p and log_beta.
            res1=res1+u/2
        x2 = Pol.base_ring()(frac(v1-(d/c)*(j+frac(v2))))
        num2 = (x2*u)._exp_series(order) ## give e^(x2u) as a polynomial
        res2=num2.multiplication_trunc(invden, order) ## multiplication of polynomial calling Flint
        if frac(v1-(d/c)*(j+frac(v2)))==0:## same thing, we change the value of b_1(0)
            res2=res2+u/2
        L1.append(res1);
        L2.append(res2);
    return([L1,L2])


def sum_for_H_value_at_k(beta,c,d,k,L1,L2):## from the functions given by power_series_values_for_sum, we 
    val=0
    for j in range (0,c):
        for r in range (0,k+1):
            val=val+(beta)^(r-1)*L1[j][r]*L2[j][k-r]
    return(val)
#[L1,L2]=power_series_values_for_sum(4,9,1000)

## Smoothing

Let $\ell\geq 5$ be a prime number and suppose that $\ell$ divide $c$, we compute the smoothed version $h_\omega^{(\ell)}$ where
$$h_\omega^{(\ell)}(t,c,d,v):=h_{\ell\omega}\biggl(\frac{t}{\ell},\frac{c}{\ell},d,(\ell v_1,v_2)\biggl)-\ell h_\omega(t,c,d,(v_1,v_2)).$$

We simply call twice the functions above.

In [1]:
def power_series_values_for_sum_smoothed(c,d,order,l,v1,v2):
    t1=cputime()
    L=power_series_values_for_sum(c,d,order,v1,v2)
    L_smoothed=power_series_values_for_sum(c/l,d,order,l*v1,v2)
    return([L,L_smoothed])
def sum_for_H_smoothed_value_at_k(beta,c,d,k,l,L,L_smoothed):#L=[L1,L2]
    t1=cputime()
    val1=sum_for_H_value_at_k(beta,c,d,k,L[0],L[1])
    val2=sum_for_H_value_at_k(beta,c/l,d,k,L_smoothed[0],L_smoothed[1])
    val_final=-(l*val1-val2/l^(k-2))
    return([val_final,val1,val2])


## Computing the integral 


Using Equation (34):
$$\textup{ord}_p(u_{\ell}^\prime(\omega))=12h_\omega^{(\ell)}(t,c,d,0,0)[t^0]+\biggl\lfloor\frac{\ell-1}{4}\biggl\rfloor$$

In [6]:
def order_p(omega,c,d,p,l):
    L_both=power_series_values_for_sum_smoothed(c,d,6,l,0,0)
    return(ZZ(sum_for_H_smoothed_value_at_k(c*omega+d,c,d,2,l,L_both[0],L_both[1])[0]+(l-1)/4))

From Equation (33), we have
$$
   \log_{\beta_p}(u_{\ell}^\prime(\omega)) =12\sum_{\substack{a,b\mod p\\a,b\neq 0,0}}\log_{\beta_p}(a+b\omega)h_\omega^{(\ell)}\biggl(t,c,d,\biggl(\frac{a}{p},\frac{b}{p}\biggl)\biggl)[t^0].
$$

In [7]:
def log_beta_help(omega_p,p,beta):
    omegap=omega_p+O(p)
    L=[[0,[1,0]]]
    for j in range (1,p^2-1):
        if j%(p+1)!=0:
            vec=pari.lindep([beta^j+O(p),1,omegap+O(p)])
            L.append([j,[ZZ(-vec[1]/vec[0]+O(p)),ZZ(-vec[2]/vec[0]+O(p))]])
        else:
            vec=ZZ(beta^j+O(p))
            L.append([j,[vec,0]])
    return(L)

def log_beta(omega,c,d,p,l,beta):
    L=log_beta_help(omega,p,beta)
    total=0
    for j in range (0,len(L)):
        vecval=L[j]
        a1,b1=[vecval[1][0],(vecval[1][1])%p]
        L_both=power_series_values_for_sum_smoothed(c,d,6,l,a1/p,b1/p)  
        total=total+vecval[0]*QQ(sum_for_H_smoothed_value_at_k(c*omega+d,c,d,2,l,L_both[0],L_both[1])[0]) 
    return(total%(p^2-1))

Finally, we compute $\log_p(u_\ell^{\prime}(\omega))$ using Equation (31)
$$\log_p(u_\ell^{\prime}(\omega))=\sum_{\substack{a,b\mod p\\a,b\neq 0,0}}\sum_{n=1}^{M'}\frac{1}{n}\sum_{j=0}^n\binom{n}{j}\frac{(-1)^{j-1}}{(a+b\omega)^j}j!p^j h_\omega^{(\ell)}(t,c,d,(\frac{a}{p},\frac{b}{p}))[t^j] +O(p^{M+1})
$$

In [10]:



def sum_log_omega(omega_p,c,d,p,l,M,v1,v2,T):#compute sum log(a+b*omega)*mu(a.b+pZ_p^2)
    val2=0
    for a in range(0,p):
        for b in range(0,p):
            if a+b!=0:
                L_values=[]
                L_both=power_series_values_for_sum_smoothed(c,d,6,l,v1+a/p,v2+b/p)
                val2=val2+log(a+omega_p*b)*QQ(sum_for_H_smoothed_value_at_k(c*omega_p+d,c,d,2,l,L_both[0],L_both[1])[0])
    return(val2)

def value_int_log(omega,c,d,p,l,M,v1,v2,T):
    time1=cputime()
    beta=c*omega+d
    omegap=T(omega)
    val_int=0
    for a in range(0,p):
        print('computation for a=',a,'and b in' ,[0,p-1])
        for b in range(0,p):
            if a+b!=0:
                L_values=[]##use series_for_H(beta,c,d,order,L1,L2) instead?
                L_both=power_series_values_for_sum_smoothed(c,d,M+10,l,a/p,b/p)##carefull with choice of v_1,v_2
               # L_both=power_series_values_for_sum_smoothed(c,d,M+10,l,v2+b/p,v1+a/p)
                for k in range (2,M+4):
                    normalisation_factor=factorial(k-2)*p^(k-2)
                    L_values.append(normalisation_factor*sum_for_H_smoothed_value_at_k(beta,c,d,k,l,L_both[0],L_both[1])[0])
                for n in range (1,M+1):
                    for j in range(0,n+1):
                        val_int=val_int+binomial(n,j)*(-1)^(j-1)*T(L_values[j])/(n*(a+b*omegap)^j)
    print(' value for',omega,' computed in ',cputime()-time1)
    return(val_int)


def value_int_log_p_adic(omega,c,d,p,l,M,v1,v2,prec,T):
    omega_p=T(omega)
    val2=sum_log_omega(omega_p,c,d,p,l,M,v1,v2,T)
    val1=value_int_log(omega,c,d,p,l,M,v1,v2,T)
    return(val1+val2,[val1,val2]) 





def get_u_omega(omega,c,d,p,l,M,beta,T):
    omega_p=T(omega)
    val_log_p=value_int_log_p_adic(omega,c,d,p,l,M,0,0,M,T)[0]
    val_log_beta=ZZ(log_beta(omega_p,c,d,p,l,beta))
    p_power=QQ(order_p(omega,c,d,p,l))
    return([(p^(p_power))*(val_log_p.exp())*T(beta)^val_log_beta,[val_log_p,val_log_beta,p_power]])


In [11]:
def get_pol_u_omega(List_val,M1,p):## return the minimal polynomial of u_\ell(\omega) using the values of u_\ell(\omega_i)
    listval_omega=[]
    power_p_total=0
    for k in range(0,len(List_val)):
        listval_omega.append(List_val[k][0])
        power_p_total+=abs(List_val[k][1][2])
    pol=1
    for u_omega in listval_omega:
        pol=pol*(x-u_omega)*(x-u_omega^(-1))
    listcoeff=pol.coefficients()
    listpower=pol.exponents()
    if abs(power_p_total)+1<M1:
        polK=0
        polK_v2=0
        for k in range(0,len(listcoeff)):## we try to recognize the coefficients using 2 methods, one using LLL and the other using Sage  
            valcoeff_p=listcoeff[k]
            coefficients=(valcoeff_p.polynomial()).coefficients()
            if len(coefficients) == 1:
                coefficients.append(0)
            x0,xd=coefficients
            p_deno=p^(power_p_total)
            find_x1=pari.lindep([p_deno*x0,1])
            find_x2=pari.lindep([p_deno*xd,1])
            x1=-(find_x1[1]/find_x1[0])
            x2=-(find_x2[1]/find_x2[0])
            val_coeff=(QQ(x1)+QQ(x2)*d)/p_deno
            polK+=x^(listpower[k])*val_coeff
            polK_v2+=x^(listpower[k])*K(valcoeff_p)
        if(polK==polK_v2):
            print('same pol with the 2 methods')
            print('for p=',p,'minimal polynomial is,',polK)
            return(polK)
        else:
            print('pol are not the same')
            print(polK,polK_v2)
            return([polK,polK_v2])
    else:
        print('not enough precision we only use one strategy to recognize the pol')
        polK_v2=0
        for k in range(0,len(listcoeff)):
            valcoeff_p=listcoeff[k]
            polK_v2+=x^(listpower[k])*K(valcoeff_p)
        return(polK_v2)
    




def computing_units(getgam,l,M1,p,T,betap): ## putting everything together
    M2=M1+floor(log(M1))
    listval_omega=[]
    t1=cputime()
    for j in range (0,len(getgam)):
        omega=getgam[j][1]
        [c,d]=getgam[j][2][1]
        List_u_info=get_u_omega(omega,c,d,p,l,M2,betap,T)
        listval_omega.append(List_u_info)
    print('final computation done in',cputime()-t1)
    pol=get_pol_u_omega(listval_omega,M1,p)
    return([pol,listval_omega])



        